In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')


In [ ]:
# import os
# path = '/content/gdrive/MyDrive/GoogleColab/Master/Thesis'
# os.chdir(path)

In [ ]:
# !pwd

In [53]:

import pandas as pd
from sklearn.preprocessing import Normalizer, LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.exceptions import ConvergenceWarning
import os
import numpy as np

from pytz import timezone
import datetime as dt

import warnings
import random

from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten

# from keras.models import load_model
from tensorflow.keras.models import load_model

import pickle

# Tắt cảnh báo
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# In[2]

TRAIN_DIR = './DATA/24_06_IS_HOT_TOMTOM/TRAIN_DATA/'
TEST_DIR = './DATA/24_06_IS_HOT_TOMTOM/TEST_DATA/'


def processing_data(data_dir):
    # TRAIN_DIR = os.path.join('.', 'is_hot_tomtom_segment_status3')
    csv_file_list = os.listdir(data_dir)
    total_df = pd.DataFrame()
    for csv_file in csv_file_list:
        temp_df = pd.read_csv(os.path.join(data_dir, csv_file), index_col=0)
        total_df = pd.concat([total_df, temp_df])

    label_encoder = LabelEncoder()
    total_df['base_LOS']= label_encoder.fit_transform(total_df['base_LOS'])
    total_df['isHot'] = label_encoder.fit_transform(total_df['isHot'])
    total_df['weather'] = label_encoder.fit_transform(total_df['weather'])

    return total_df

def get_period_df(parameters, total_df):
    period_df = total_df.copy()
    for key, value in parameters.items():
        if value is not None:
            period_df = period_df.loc[period_df[key].isin(value)]
    return period_df

In [2]:
total_df = processing_data(TRAIN_DIR)
test_df = processing_data(TEST_DIR)

In [3]:
total_df.head(5)

,period,segment_id,date,weekday,tomtom_velocity,base_LOS,isHot,weather,temperature,district,lat,lng,is_morning,duration_velocity,duration_LOS
0,period_19_00,23795,2023-05-13,5,23,3,0,0,27,Quận Gò Vấp,10.830985,106.677322,0,1,1
1,period_19_00,60703,2023-05-13,5,23,3,0,0,27,Quận Gò Vấp,10.831059,106.677386,0,1,1
2,period_19_00,23794,2023-05-13,5,23,3,0,0,27,Quận Gò Vấp,10.830916,106.677520,0,1,1
3,period_19_00,60704,2023-05-13,5,23,3,0,0,27,Quận Gò Vấp,10.830894,106.677499,0,1,1
4,period_19_00,60538,2023-05-13,5,23,3,0,0,27,Quận Gò Vấp,10.830924,106.677335,0,1,1


In [22]:
def processing_data2(period_df, cluster_colums = ['tomtom_velocity', 'duration_velocity']):
    period_df_time = period_df.copy()
    for column_item in period_df_time.columns:
        if column_item not in cluster_colums:
            period_df_time.drop([column_item], axis=1, inplace = True)
    return period_df_time

In [23]:
def create_sliding_window(data, window_size):
    X, Y = [], []
    for i in range(len(data) - window_size):
        window = data[i:(i + window_size), :]
        X.append(window)
        Y.append(data[i + window_size, :])
    return np.array(X), np.array(Y)

In [24]:
def train_test_split_func(data_X, data_Y):
    train_ratio = 0.7
    train_size = int(len(data_X) * train_ratio)

    train_X, test_X = data_X[:train_size], data_X[train_size:]
    train_Y, test_Y = data_Y[:train_size], data_Y[train_size:]

    return train_X, test_X, train_Y, test_Y


In [25]:

def get_traning_test_data(data_df, seg_id_item):
#     # Tại một quận cụ thể
#     parameters = {
#         "period": ["period_16_55"],
#         "weekday": [1],
#         "district": ["Thành phố Thủ Đức"],
#     #     "is_morning": [1]
#     }
#     period_df = get_period_df(parameters, data_df)

    train_df = pd.DataFrame([])
    temp_seg_df = data_df.loc[data_df.segment_id == seg_id_item].copy()

    scaler = MinMaxScaler(feature_range=(0, 1))
    temp_seg_df[['tomtom_velocity', 'duration_velocity']] = scaler.fit_transform(temp_seg_df[['tomtom_velocity', 'duration_velocity']])

    with open(f'./SCALERS/{seg_id_item}.pkl', 'wb') as file:
        pickle.dump(scaler, file)

    train_X, train_Y = [], []
    test_X, test_Y = [], []
    for date_item in temp_seg_df.date.unique():
        temp_date_df = temp_seg_df.loc[temp_seg_df.date == date_item]
        for is_morning_item in temp_date_df.is_morning.unique():
            temp_df = temp_date_df.loc[temp_date_df.is_morning == is_morning_item]
            if temp_df.shape[0] == 0:
                continue
            train_df = processing_data2(temp_df)
            train_df = np.array(train_df)
            temp_X, temp_Y = create_sliding_window(train_df, 10)

            temp_train_X, temp_test_X, temp_train_Y, temp_test_Y = train_test_split_func(temp_X, temp_Y)

            if len(train_X) == 0:
                train_X = temp_train_X
                train_Y = temp_train_Y

                test_X = temp_test_X
                test_Y = temp_test_Y

            else:
                train_X = np.concatenate((train_X, temp_train_X), axis = 0)
                train_Y = np.concatenate((train_Y, temp_train_Y), axis = 0)

                test_X = np.concatenate((test_X, temp_test_X), axis = 0)
                test_Y = np.concatenate((test_Y, temp_test_Y), axis = 0)

    return train_X, train_Y, test_X, test_Y


In [26]:
def processing_data_needed_to_predict(data_df, seg_id_item):
#     # Tại một quận cụ thể
#     parameters = {
#         "period": ["period_16_55"],
#         "weekday": [1],
#         "district": ["Thành phố Thủ Đức"],
#     #     "is_morning": [1]
#     }
#     period_df = get_period_df(parameters, data_df)

    train_df = pd.DataFrame([])
    temp_seg_df = data_df.loc[data_df.segment_id == seg_id_item].copy()

    with open(f'./SCALERS/{seg_id_item}.pkl', 'rb') as file:
        scaler = pickle.load(file)
    # scaler = MinMaxScaler(feature_range=(0, 1))
    # temp_seg_df[['tomtom_velocity', 'duration_velocity']] = scaler.fit_transform(temp_seg_df[['tomtom_velocity', 'duration_velocity']])

    temp_seg_df[['tomtom_velocity', 'duration_velocity']] = scaler.transform(temp_seg_df[['tomtom_velocity', 'duration_velocity']])



    train_X, train_Y = [], []
    for date_item in temp_seg_df.date.unique():
        temp_date_df = temp_seg_df.loc[temp_seg_df.date == date_item]
        for is_morning_item in temp_date_df.is_morning.unique():
            temp_df = temp_date_df.loc[temp_date_df.is_morning == is_morning_item]
            if temp_df.shape[0] == 0:
                continue
            train_df = processing_data2(temp_df)
            train_df = np.array(train_df)
            temp_train_X, temp_train_Y = create_sliding_window(train_df, 10)

            if len(train_X) == 0:
                train_X = temp_train_X
                train_Y = temp_train_Y

            else:
                train_X = np.concatenate((train_X, temp_train_X), axis = 0)
                train_Y = np.concatenate((train_Y, temp_train_Y), axis = 0)

    return train_X, train_Y

In [27]:
def create_multi_output_model(out_size = 2, window_size = 10):
    # Xây dựng mô hình hỗn hợp
    model = Sequential()
    model.add(LSTM(50, input_shape=(window_size, out_size))) #train_X.shape[2] = out_size
    model.add(Dense(50, activation='relu'))
    model.add(Dense(out_size))  # Số lượng đầu ra bằng số lượng thuộc tính (ở đây là 3)
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

In [28]:
def create_hybrid_model(out_size = 2, window_size = 10):
    model = Sequential()
    model.add(LSTM(50, input_shape=(window_size, out_size))) #train_X.shape[2] = out_size
    model.add(Flatten())
    model.add(Dense(50, activation='relu'))
    model.add(Dense(out_size))
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

## TEST MODEL WITH SINGLE SEGMENT

In [29]:
seg_id_item = 23795
train_X, train_Y, test_X, test_Y = get_traning_test_data(total_df, seg_id_item)

In [30]:
multi_output_model = create_multi_output_model(train_X.shape[2])

In [31]:
# Huấn luyện mô hình
multi_output_model.fit(train_X, train_Y, epochs=10, batch_size=16)

Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0099
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0068
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0065
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0063
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0060
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0058


In [32]:
# Đánh giá mô hình trên tập kiểm tra
train_loss = multi_output_model.evaluate(train_X, train_Y, verbose=0)
print('Train loss:', train_loss)
test_loss = multi_output_model.evaluate(test_X, test_Y, verbose=0)
print('Test loss:', test_loss)

Train loss: 0.0056926836259663105
Test loss: 0.007969370111823082


In [33]:
val_X, val_Y = processing_data_needed_to_predict(test_df, seg_id_item)

In [ ]:
loaded_model = load_model(f"./MODELS/MultiOutputModels/{seg_id_item}.h5")
val_loss = loaded_model.evaluate(val_X, val_Y, verbose=0)
print(val_loss)

In [ ]:
with open(f'./SCALERS/{seg_id_item}.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [ ]:
predicted_data = scaler.inverse_transform(predicted_data)

In [ ]:
predicted_data

In [ ]:
val_Y =  scaler.inverse_transform(val_Y)
val_Y

## CREATE MODEL FOR EACH SEGMENT

### Create model with training model function

In [40]:
def training_model(seg_id_item, is_hybrid_model = False):
    train_X, train_Y, test_X, test_Y = get_traning_test_data(total_df, seg_id_item)
    if is_hybrid_model:
        model = create_multi_output_model(train_X.shape[2])
    else:
        model = create_hybrid_model(train_X.shape[2])

    model.fit(train_X, train_Y, epochs=10, batch_size=16)
    path_saved_model = f"./MODELS/HybridModels/{seg_id_item}.h5" if is_hybrid_model else f"./MODELS/MultiOutputModels/{seg_id_item}.h5"
    model.save(path_saved_model)

    return model

In [54]:
training_model(seg_id_item)

Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0089
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0066
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0063
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0060
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0060
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0060


In [42]:
val_X, val_Y = processing_data_needed_to_predict(test_df, seg_id_item)

In [47]:
!pip install --upgrade tensorflow

  Using cached tensorflow-2.13.0rc2-cp39-cp39-macosx_12_0_arm64.whl (2.0 kB)
     |████████████████████████████████| 189.3 MB 7.3 MB/s eta 0:00:011    |█████████████▋                  | 80.2 MB 12.9 MB/s eta 0:00:09


  Attempting uninstall: tensorflow-macos
    Found existing installation: tensorflow-macos 2.13.0rc1
    Uninstalling tensorflow-macos-2.13.0rc1:
      Successfully uninstalled tensorflow-macos-2.13.0rc1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.13.0rc1
    Uninstalling tensorflow-2.13.0rc1:
      Successfully uninstalled tensorflow-2.13.0rc1
You should consider upgrading via the '/Users/xaviele/Documents/MASTER/THEORY/Example_UrbanTrafficCondition_Sharding/env/bin/python3 -m pip install --upgrade pip' command.


###

In [ ]:
segments_list = test_df.segment_id.unique()

### MultiOutputModels
using to predict the value of the single feature

In [56]:
MultiOutputModels_loss_result_dict = []
count = 0
for segment_item in test_df.segment_id.unique():
    print("--------segment_item: ", segment_item)
    print("------count:", count)
    count +=1
    model = training_model(segment_item)
    
    train_X, train_Y, test_X, test_Y = get_traning_test_data(total_df, segment_item)
    val_X, val_Y = processing_data_needed_to_predict(test_df, segment_item)

    train_loss = model.evaluate(train_X, train_Y, verbose=0)
    test_loss = model.evaluate(test_X, test_Y, verbose=0)
    val_loss = model.evaluate(val_X, val_Y, verbose=0)

    MultiOutputModels_loss_result_dict.append([segment_item, train_loss, test_loss, val_loss])

--------segment_item:  23795
------count: 0
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0173
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0069
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0066
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0063
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0059
--------segment_item:  60703
------count: 1
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0117
Epoch 2/10
105/105 [

105/105 [==============================] - 0s 2ms/step - loss: 0.0062
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0060
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0059
--------segment_item:  47580
------count: 10
Epoch 1/10
105/105 [==============================] - 1s 3ms/step - loss: 0.0167
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0036
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0035
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 

105/105 [==============================] - 1s 2ms/step - loss: 0.0164
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0038
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0036
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0035
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0034
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0035
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0033
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0034
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0033
--------segment_item:  8197
------count: 20
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0310
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0

105/105 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0083
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0080
--------segment_item:  35896
------count: 29
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0299
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0090
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0084
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0083
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0084
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 

105/105 [==============================] - 0s 2ms/step - loss: 0.0094
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0085
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0084
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0083
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0080
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0080
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0080
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0080
--------segment_item:  70493
------count: 39
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0304
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0084
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 

...

105/105 [==============================] - 0s 2ms/step - loss: 0.0065
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0065
--------segment_item:  74644
------count: 487
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0302
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0173
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0164
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0155
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0149
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0144
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0143
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0137
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0138
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss

105/105 [==============================] - 0s 2ms/step - loss: 0.0156
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0153
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0147
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0144
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0138
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0137
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0136
--------segment_item:  59366
------count: 497
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0274
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0171
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0160
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0154
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss:

105/105 [==============================] - 0s 2ms/step - loss: 0.0069
--------segment_item:  33185
------count: 506
Epoch 1/10
105/105 [==============================] - 2s 2ms/step - loss: 0.0358
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0086
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0080
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0076
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0077
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0075
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0070
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0070
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0070
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0069
--------segment_item:  42427
------count: 507
Epoch 1/10
105/105 [=======

105/105 [==============================] - 0s 2ms/step - loss: 0.0183
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0185
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0181
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0180
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0182
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0178
--------segment_item:  34358
------count: 516
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0310
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0185
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0184
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0182
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0182
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss:

Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0179
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0099
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0099
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0097
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0095
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0095
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0094
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0093
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0093
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0093
--------segment_item:  82237
------count: 526
Epoch 1/10
105/105 [==============================] - 2s 2ms/step - loss: 0.0190
Epoch 2/10
105/105 [==============================] - 0s 2ms/s

In [58]:
header = ["segment_id", "train_loss", "test_loss", "val_loss"]
MultiOutputModels_loss_result_df = pd.DataFrame(MultiOutputModels_loss_result_dict, columns=header)


### Evaluate to performance of model with test data


In [20]:

MultiOutputModels_loss_result_dict = []
count = 0
for segment_item in test_df.segment_id.unique():
    print("count:", count)
    count += 1
    train_X, train_Y, test_X, test_Y = get_traning_test_data(total_df, segment_item)
    val_X, val_Y = processing_data_needed_to_predict(test_df, segment_item)

    loaded_model = load_model(f"./MODELS/MultiOutputModels/{segment_item}.h5")

    train_loss = loaded_model.evaluate(train_X, train_Y, verbose=0)
    test_loss = loaded_model.evaluate(test_X, test_Y, verbose=0)
    val_loss = loaded_model.evaluate(val_X, val_Y, verbose=0)

    MultiOutputModels_loss_result_dict.append([segment_item, train_loss, test_loss, val_loss])


header = ["segment_id", "train_loss", "test_loss", "val_loss"]
MultiOutputModels_loss_result_df = pd.DataFrame(MultiOutputModels_loss_result_dict, columns=header)







count: 0


count: 1


count: 2


count: 3


count: 4


...

count: 331


count: 332


count: 333


count: 334


KeyboardInterrupt: 

In [59]:
MultiOutputModels_loss_result_df.to_csv("./LOG/MultiOutputModelLog.csv")

### HybridModels
Using to predict the value of multiple features that is dependent each other

In [60]:
HybridModels_loss_result_dict = []
count = 0
for segment_item in test_df.segment_id.unique():
    print("--------segment_item: ", segment_item)
    print("---------count: ", count)
    count += 1
    model = training_model(segment_item, True)
    train_X, train_Y, test_X, test_Y = get_traning_test_data(total_df, segment_item)
    val_X, val_Y = processing_data_needed_to_predict(test_df, segment_item)

    train_loss = model.evaluate(train_X, train_Y, verbose=0)
    test_loss = model.evaluate(test_X, test_Y, verbose=0)
    val_loss = model.evaluate(val_X, val_Y, verbose=0)

    HybridModels_loss_result_dict.append([segment_item, train_loss, test_loss, val_loss])

--------segment_item:  23795
---------count:  0
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0142
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0071
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0063
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0060
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0059
--------segment_item:  60703
---------count:  1
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0091
Epoch 2/10
1

105/105 [==============================] - 0s 2ms/step - loss: 0.0061
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0060
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0060
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0059
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0059
--------segment_item:  47580
---------count:  10
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0248
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0044
Epoch 3/10
105/105 [==============================] - 0s 3ms/step - loss: 0.0038
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0036
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0036
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - lo

105/105 [==============================] - 1s 2ms/step - loss: 0.0215
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0040
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0038
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0037
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0036
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0035
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0034
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0035
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0034
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0037
--------segment_item:  8197
---------count:  20
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0231
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - los

105/105 [==============================] - 0s 2ms/step - loss: 0.0084
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0080
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0081
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0081
--------segment_item:  35896
---------count:  29
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0233
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0090
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0084
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0083
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - lo

105/105 [==============================] - 1s 2ms/step - loss: 0.0238
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0085
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0084
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0081
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0080
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0082
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0079
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0080
--------segment_item:  70493
---------count:  39
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0530
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - lo

...

105/105 [==============================] - 0s 2ms/step - loss: 0.0067
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0066
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0065
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0064
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0064
--------segment_item:  16084
---------count:  485
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0393
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0083
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0075
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0072
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0069
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0069
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - l

Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0361
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0176
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0165
Epoch 4/10
105/105 [==============================] - 0s 3ms/step - loss: 0.0158
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0150
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0147
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0142
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0143
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0140
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0138
--------segment_item:  39116
---------count:  495
Epoch 1/10
105/105 [==============================] - 2s 2ms/step - loss: 0.0230
Epoch 2/10
105/105 [==============================] - 0s 2

105/105 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0072
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0072
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0069
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0069
--------segment_item:  31200
---------count:  504
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0410
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0090
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0085
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0077
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0077
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0074
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - l

Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0255
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0189
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0186
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0183
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0183
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0184
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0182
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0181
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0180
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0181
--------segment_item:  24374
---------count:  514
Epoch 1/10
105/105 [==============================] - 1s 2ms/step - loss: 0.0304
Epoch 2/10
105/105 [==============================] - 0s 2

105/105 [==============================] - 0s 2ms/step - loss: 0.0093
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0094
Epoch 8/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0093
Epoch 9/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0091
Epoch 10/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0092
--------segment_item:  34500
---------count:  523
Epoch 1/10
105/105 [==============================] - 2s 2ms/step - loss: 0.0144
Epoch 2/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0099
Epoch 3/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0100
Epoch 4/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0096
Epoch 5/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0094
Epoch 6/10
105/105 [==============================] - 0s 2ms/step - loss: 0.0095
Epoch 7/10
105/105 [==============================] - 0s 2ms/step - l

In [61]:
header = ["segment_id", "train_loss", "test_loss", "val_loss"]
HybridModels_loss_result_df = pd.DataFrame(HybridModels_loss_result_dict, columns=header)
HybridModels_loss_result_df.to_csv("./LOG/HybridModelsLog.csv")

## Function to predict velocity and duration

In [66]:
from sklearn.metrics import mean_squared_error

In [67]:
def predict_function(seg_id, features_input, target_y, is_hybrid_model = False):
    if is_hybrid_model: 
        loaded_model = load_model(f"./MODELS/HybridModels/{seg_id_item}.h5")
    else:
        loaded_model = load_model(f"./MODELS/MultiOutputModels/{seg_id_item}.h5")
    
    with open(f'./SCALERS/{seg_id}.pkl', 'rb') as file:
        scaler = pickle.load(file)
        
    predicted_data = loaded_model.predict(features_input)
    
    mse = mean_squared_error(target_y, predicted_data)
    print("mse: ", mse)
    
    predicted_data = scaler.inverse_transform(predicted_data)
    
    return predicted_data
    

In [62]:
seg_id_item = 23795
val_X, val_Y = processing_data_needed_to_predict(test_df, seg_id_item)
loaded_model = load_model(f"./MODELS/MultiOutputModels/{seg_id_item}.h5")
val_loss = loaded_model.evaluate(val_X, val_Y, verbose=0)
print(val_loss)


0.005583765916526318


In [68]:
predicted_data = predict_function(seg_id_item, val_X, val_Y)

12/12 [==============================] - 1s 6ms/step
mse:  0.005583765853819879


In [65]:
print(predicted_data)

[[24.522842   1.3054733]
 [23.919195   1.279362 ]
 [23.486053   1.443916 ]
 [24.035826   1.4047346]
 [24.300697   1.5571222]
 [24.450676   1.8609997]
 [24.531006   2.2753203]
 [23.684698   1.9789226]
 [23.072212   1.9606225]
 [23.116182   1.7748584]
 [22.389524   1.6210473]
 [21.875721   1.6868644]
 [21.97037    1.5582945]
 [21.973515   1.6540012]
 [23.124672   1.5657212]
 [23.767221   1.6735408]
 [24.577085   1.5791459]
 [24.41203    1.4958992]
 [24.640366   1.4383053]
 [25.777699   1.4417176]
 [26.194843   1.4224571]
 [26.429018   1.5685462]
 [26.555794   1.9095812]
 [29.584106   1.7919555]
 [28.449001   1.6932789]
 [28.345314   1.6317931]
 [21.828125   1.345191 ]
 [21.119686   1.4747404]
 [21.470016   1.4211969]
 [21.635725   1.5577253]
 [22.178427   1.4895892]
 [22.461184   1.6154137]
 [22.35395    1.5184673]
 [22.219698   1.6220794]
 [22.092758   1.8718115]
 [22.497692   1.7075251]
 [22.690634   1.772726 ]
 [22.141373   1.6110756]
 [22.511314   1.5255357]
 [21.995514   1.4343965]


In [ ]:
mse = mean_squared_error(test_target, predictions)
print("Mean Squared Error (MSE):", mse)